In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
import feature_manager as fma
import classifier.multi_dnn_classifier as dnn
from random import randint
from keras import callbacks, losses
import visualizer
from keras.utils import np_utils
from sklearn.metrics import classification_report, confusion_matrix
import importlib
import tr_utils

2023-01-22 12:37:37.558701: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-22 12:37:37.558806: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-22 12:37:37.558819: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-01-22 12:37:38.658228: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-22 12:37:38.658275: I tensorflow/compiler/xla/stream_exe

In [ ]:
importlib.reload(fma)

In [19]:
metric_list = [
    "loss",
    "accuracy",
    "precision",
    "recall",
    "precision-0.65",
    "recall-0.65",
    "precision-0.80",
    "recall-0.80",
    "precision-0.95",
    "recall-0.95"
]

symbol = "BTCUSDT"
trade_tf = "1h"
granular_tf = "1m"

tp = 0.013
sl = 0.013
md = 7

fm = fma.FeatureManager(
    target_col="trade_signal"
)

fm.import_data(symbol=symbol,trade_timeframe=trade_tf,granular_timeframe=granular_tf)

fm.prepare_trade_forward_data(
    take_profit_rate=tp,
    stop_loss_rate=sl,
    max_duration=md,
)

Imported ../data/BTCUSDT-1h.csv with 46982 rows
Imported ../nocommit/BTCUSDT-1m.csv with 2817999 rows
Scanning 7 future timeframes to build trade signal: 
1, 2, 3, 4, 5, 6, 7, 
Label producing completed. 
 Value counts:
2    15959
0    15689
1    15334
Name: trade_signal, dtype: int64


In [20]:
fm.build_features(lags=30)

Calculating features...

Adding feature: returns, dir, sma, boll, boll7, boll14, boll21, min, min7, min14, min21, max, max7, max14, max21, mom, mom7, mom14, mom21, vol, vol7, vol14, vol21, obv, mfi7, mfi14, mfi21, rsi7, rsi14, rsi21, adx7, adx14, adx21, roc, roc7, roc14, roc21, atr7, atr14, atr21, bop, ad, adosc, trange, ado, willr7, willr14, willr21, dx7, dx14, dx21, trix, ultosc, high, low, 

Normalizing feature: returns_lag_1, returns_lag_2, returns_lag_3, returns_lag_4, returns_lag_5, returns_lag_6, returns_lag_7, returns_lag_8, returns_lag_9, returns_lag_10, returns_lag_11, returns_lag_12, returns_lag_13, returns_lag_14, returns_lag_15, returns_lag_16, returns_lag_17, returns_lag_18, returns_lag_19, returns_lag_20, returns_lag_21, returns_lag_22, returns_lag_23, returns_lag_24, returns_lag_25, returns_lag_26, returns_lag_27, returns_lag_28, returns_lag_29, returns_lag_30, dir_lag_1, dir_lag_2, dir_lag_3, dir_lag_4, dir_lag_5, dir_lag_6, dir_lag_7, dir_lag_8, dir_lag_9, dir_lag_10,

In [24]:
importlib.reload(dnn)

<module 'classifier.multi_dnn_classifier' from '/home/andy/CryptoTradingPlatform/TraderRobot/project/classifier/multi_dnn_classifier.py'>

In [25]:
results_list = []
results = dnn.evaluate_classifier_k_folds(
    hu=100,
    fm=fm,
    fold_number=5,
    batch_size=20,
    set_class_weight=True,
    save_check_point=False,
    early_stopping=True,
    dropout=True,
    dropout_rate=0.3,
    shuffle_when_train=False,
    gpu=False,
    metrics=metric_list,
    write_to_file=True)
results_list.append(results)


DATA PREPARATION PARAMS:
trade_timeframe:1h
take_profit_rate:0.013
stop_loss_rate:0.013
max_duration:7
lags:30
fold_number:5
train_size:0.7
val_size:0.15
categorical_label:True
rebalance:None
DATA:
Total rows: 46803
Label 0: 15686(33.51%)
Label 1: 15252(32.59%)
Label 2: 15865(33.9%)
Splitting the data...
Proccesing fold 1

FOLD 1
Train: 6552, Validation: 1404, Test: 1404

Train:
Label 0: 597(9.11%)
Label 1: 3058(46.67%)
Label 2: 2897(44.22%)

Validation:
Label 0: 574(40.88%)
Label 1: 412(29.34%)
Label 2: 418(29.77%)

Test:
Label 0: 532(37.89%)
Label 1: 381(27.14%)
Label 2: 491(34.97%)
Proccesing fold 2

FOLD 2
Train: 6552, Validation: 1404, Test: 1404

Train:
Label 0: 3414(52.11%)
Label 1: 1598(24.39%)
Label 2: 1540(23.5%)

Validation:
Label 0: 221(15.74%)
Label 1: 562(40.03%)
Label 2: 621(44.23%)

Test:
Label 0: 673(47.93%)
Label 1: 342(24.36%)
Label 2: 389(27.71%)
Proccesing fold 3

FOLD 3
Train: 6552, Validation: 1404, Test: 1404

Train:
Label 0: 2882(43.99%)
Label 1: 1748(26.68%)


In [ ]:
metric_list = [
    "loss",
    "accuracy",
    "precision",
    "recall",
    "precision-0.65",
    "recall-0.65",
    "precision-0.80",
    "recall-0.80",
    "precision-0.95",
    "recall-0.95"
]

In [ ]:
results_list = []
results = dnn.evaluate_classifier(
    hu=100,
    fm=fm,
    laps=1,
    batch_size=20,
    set_class_weight=False,
    save_check_point=False,
    early_stopping=True,
    shuffle_before_split=False,
    dropout=True,
    dropout_rate=0.3,
    gpu=False,
    metrics=metric_list,
    write_to_file=True)
results_list.append(results)

In [ ]:
results_df = pd.DataFrame(results)

In [ ]:
results_df.to_csv("../out/evaluate.csv")

In [ ]:
classifier = dnn.MultiDNNClassifer()

dataset = classifier.prepare_data(
    data = fm.df,
    cols = fm.cols,
    shuffle_before_split= False,
    categorical_label=True,
    target_col="trade_signal"
)

initial_bias = tr_utils.init_imbalanced_bias(
    y_train=dataset[1]
)

classifier.configure(
    hu = 100, 
    dropout=True, 
    dropout_rate = 0.3,
    input_dim=len(fm.cols),
    output_bias=initial_bias,
    class_num=3
)